# Prepare dataset


In [1]:
from datasets import load_dataset

# loading the V2 dataset
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")

/mnt/kalista/git-cuongpiger/langchain-labs/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
amnesty_subset = amnesty_qa["eval"].select(range(2))

In [3]:
amnesty_subset.to_pandas()

,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[In recent years, there has been increasing pr..."


# Initialize model


In [4]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [5]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from ragas import evaluate

In [6]:
model_name = "llama3.1:8b"
base_url = "http://61.28.230.60:11434"
langchain_llm = ChatOllama(model=model_name, base_url=base_url)
langchain_embeddings = OllamaEmbeddings(model=model_name, base_url=base_url)

# Evaluate


In [7]:
result = evaluate(
    amnesty_subset,
    metrics=[context_precision, faithfulness, answer_relevancy, context_recall],
    llm=langchain_llm,
    embeddings=langchain_embeddings,
)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]/mnt/kalista/git-cuongpiger/langchain-labs/.venv/lib/python3.10/site-packages/ragas/metrics/_answer_relevance.py:107: RuntimeWarning: invalid value encountered in divide
  np.dot(gen_question_vec, question_vec.T).reshape(
Evaluating:  25%|██▌       | 2/8 [00:19<00:49,  8.23s/it]/mnt/kalista/git-cuongpiger/langchain-labs/.venv/lib/python3.10/site-packages/ragas/metrics/_answer_relevance.py:107: RuntimeWarning: invalid value encountered in divide
  np.dot(gen_question_vec, question_vec.T).reshape(
Evaluating:  75%|███████▌  | 6/8 [00:25<00:04,  2.29s/it]Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt n_l_i_statement_prompt failed to parse output: 

In [8]:
print(result)

{'context_precision': 1.0000, 'faithfulness': nan, 'answer_relevancy': nan, 'context_recall': 0.9286}
